# This code to generate answer and retrieved documents column with propositional for Hallubench dataset using Simple RAG pipeline with Mistral locally using Langchain Ollama

In [ ]:
#@title { vertical-output: true}
# Install ollama: The Ollama API is now available at 127.0.0.1:11434
! curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
#@title { vertical-output: true}
import subprocess
process = subprocess.Popen("ollama serve", shell=True)
# credit: https://stackoverflow.com/questions/78437376/run-ollama-run-llama3-in-colab-raise-err-error-could-not-connect-to-ollama

# download the Mistral model
!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling ff82381e2bea... 100% ▕▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling 491dfa501e59... 100% ▕▏  801 B                         
pulling ed11eda7790d... 100% ▕▏   30 B                         
pulling 42347cd80dc8... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
#@title { vertical-output: true}
!pip install -q langchain
!pip install -U -q langchain-community
!pip install -U langchain-huggingface sentence-transformers
!pip install -q chromadb
!pip install datasets
!pip install -q -U ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67

In [ ]:
#@title { vertical-output: true}
import pandas as pd
from datasets import Dataset, Features, Sequence, Value
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from pydantic import BaseModel, Field
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Initialize Ollama LLM and Embeddings
llm: BaseLanguageModel = Ollama(model="mistral", temperature = 0.1)

In [ ]:
#@title { vertical-output: true}
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

<ipython-input-7-f0e2007b65f4>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#@title { vertical-output: true}
# Load the dataset from a local CSV file

df = pd.read_csv('/content/halubench_short.csv')

# Ensure the dataset has the required columns: 'question', 'contexts', 'ground_truth'
required_columns = ['question', 'contexts', 'ground_truth']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"The dataset must contain these columns: {required_columns}")


In [ ]:
import torch
import json
# Load propositional chunking model
model_name = "chentong00/propositionizer-wiki-flan-t5-large"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
prop_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
def propositional_chunking(text):
    input_text = f"Title: . Section: . Content: {text}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = prop_model.generate(input_ids.to(device), max_new_tokens=8000).cpu()
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #try:
    #    prop_list = json.loads(output_text)
    #except:
    #    prop_list = []
    #    print("[ERROR] Failed to parse output text as JSON.")
    return output_text

In [ ]:
# Apply propositional chunking to the entire 'contexts' column
print("Applying propositional chunking to the dataset...")
tqdm.pandas(desc="Chunking contexts")
df['contexts'] = df['contexts'].progress_apply(lambda x: propositional_chunking(str(x)))

Applying propositional chunking to the dataset...


Chunking contexts: 100%|██████████| 60/60 [26:52<00:00, 26.88s/it]


In [ ]:
#@title { vertical-output: true}
def calculation(row, llm=llm, embed_model=embed_model):
    # Convert 'contexts' to string if it's not
    context = str(row['contexts'])
    question = str(row['question'])

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=20)
    all_splits = text_splitter.split_text(context)

    # Create a vectorstore from the propositional chunks
    vectorstore = Chroma.from_texts(texts=all_splits, embedding=embed_model)

    # Define the prompt template
    prompt_template = """
    You are a helpful AI assistant. Use the following pieces of information to answer the user's question. Provide a concise answer in one sentence only. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context: {context}
    Question: {question}
    """

    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    # Create the RAG pipeline
    rag_pipeline = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True,
        chain_type_kwargs={'prompt': prompt}
    )

    result = rag_pipeline({"query": question})

    # Concatenate retrieved documents
    retrieved_docs = "\n\n".join([doc.page_content for doc in result['source_documents']])

    # Return the result and retrieved documents
    return pd.Series({
        'answer': result['result'],
        'retrieved_documents': retrieved_docs
    })

# Apply the calculation function to each row with a progress bar
tqdm.pandas(desc="Processing rows")
df[['answer', 'retrieved_documents']] = df.progress_apply(calculation, axis=1)

print("Processing complete!")

Processing rows: 100%|██████████| 60/60 [00:28<00:00,  2.12it/s]

Processing complete!


In [ ]:
# Select required columns for the Hugging Face dataset
df_hf = df[required_columns + ['answer', 'retrieved_documents']]

# Define the schema for the Hugging Face dataset
features = Features({
    'question': Value('string'),
    'contexts': Sequence(Value('string')),
    'ground_truth': Value('string'),
    'answer': Value('string'),
    'retrieved_documents': Value('string'),
})

In [ ]:
# Save the DataFrame with generated answers to a CSV file
output_csv_path = 'halubench_short_answers-SimpleRAG_Ollama_Retriver_Propositional.csv'
df.to_csv(output_csv_path, index=False)